<a href="https://colab.research.google.com/github/Koaxz/colab/blob/1/youtube_stream_to_kick%2Bvkplay_streamlink_WORK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!apt-get install streamlink ffmpeg
!pip install --upgrade streamlink
!apt-get install ffmpeg
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=55

In [2]:
import gradio as gr
import subprocess

def start_restream(youtube_url, kick_key, kick_rtmp, vk_key, vk_rtmp):
    try:
        # Get video and audio stream URLs
        video_url = subprocess.check_output(f"yt-dlp --get-url -f bestvideo '{youtube_url}' 2> yt-dlp_errors.log", shell=True, text=True).strip()
        audio_url = subprocess.check_output(f"yt-dlp --get-url -f bestaudio '{youtube_url}' 2> yt-dlp_errors.log", shell=True, text=True).strip()

        # Attempt to get Kick server IP
        try:
            kick_server_ip = subprocess.check_output("ping -c 1 live.kick.com | grep -oP '\\\\d+\\\\.\\\\d+\\\\.\\\\d+\\\\.\\\\d+'", shell=True, text=True).strip()
        except subprocess.CalledProcessError:
            kick_server_ip = "live.kick.com"

        # Start Kick restream using FFmpeg and yt-dlp
        kick_command = f"ffmpeg -re -i '{video_url}' -i '{audio_url}' -c:v libx264 -preset veryfast -b:v 3000k -maxrate 3000k -bufsize 6000k -pix_fmt yuv420p -g 50 -c:a aac -b:a 160k -ac 2 -ar 44100 -f flv {kick_rtmp}/{kick_server_ip}/{kick_key}"
        kick_process = subprocess.Popen(kick_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

        # Start VK Play Live restream using FFmpeg and yt-dlp
        vk_command = f"ffmpeg -re -i '{video_url}' -i '{audio_url}' -c:v libx264 -preset veryfast -b:v 3000k -maxrate 3000k -bufsize 6000k -pix_fmt yuv420p -g 50 -c:a aac -b:a 160k -ac 2 -ar 44100 -f flv {vk_rtmp}/{vk_key}"
        vk_process = subprocess.Popen(vk_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

        kick_output, kick_error = kick_process.communicate()
        vk_output, vk_error = vk_process.communicate()

        if kick_process.returncode != 0 or vk_process.returncode != 0:
            with open("yt-dlp_errors.log", "r") as f:
                yt_dlp_errors = f.read()
            return "Error starting restream:\\n\\nKick: " + kick_error.decode('utf-8') + "\\nVK Play Live: " + vk_error.decode('utf-8') + "\\n\\nyt-dlp errors:\\n" + yt_dlp_errors
        else:
            return "Restream successfully started on Kick and VK Play Live!"
    except Exception as e:
        return "An error occurred: " + str(e)

# Gradio interface
iface = gr.Interface(
    fn=start_restream,
    inputs=[
        gr.Textbox(label="YouTube Stream Link"),
        gr.Textbox(label="Kick Stream Key"),
        gr.Textbox(label="Kick RTMP Stream URL", value="rtmp://live.kick.com/app"),
        gr.Textbox(label="VK Play Live Stream Key"),
        gr.Textbox(label="VK Play Live RTMP Stream URL", value="rtmp://vsu.mycdn.me/input")
    ],
    outputs=gr.Textbox(label="Result"),
    title="Restream YouTube to Kick and VK Play Live",
    description="Enter the YouTube stream link, stream keys, and RTMP stream URLs for Kick and VK Play Live to start the restream."
)
# Launch interface

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://330885d9e82a32cf22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
